# Setup

There is a hidden initialization cell below. It should run automatically and print, but if it does not, please run it.

In [18]:
%matplotlib inline

print('Loading libraries... Please wait.')

from IPython.display import display, clear_output
import ipywidgets as widgets
import random
import sys
from sklearn.metrics import confusion_matrix, accuracy_score

from esper.prelude import *
from esper.stdlib import *
import esper.face_embeddings as face_embeddings


WIDGET_STYLE_ARGS = {'description_width': 'initial'}

def load_samples():
    # load up samples
    with open(os.path.join(PATH, FILE_NAME), 'rb') as f:
        samples = pickle.load(f)
    
    result = []
    for s in samples[START_INDEX:END_INDEX]:
        video, min_frame, max_frame = s
        query_result = {
            'type': 'flat', 'label': '', 
            'elements': [{
                'objects': [], 
                'min_frame': min_frame,
                'max_frame': max_frame,
                'video': video
            }]
        }
        result.append(query_result)
    result_obj = {'type': '', 'count': 0, 'result': result}
    
    return result_obj

def label_segments(result_obj):
    widg = esper_widget(
        result_obj, jupyter_keybindings=True,
        results_per_page=100
    )
    
    calculate_precision_button = widgets.Button(
        layout=widgets.Layout(width='auto'),
        style=WIDGET_STYLE_ARGS,
        description='Calculate Accuracy',
        disabled=False,
        button_style=''
    )
    def on_calculate(b):
        precision = len(widg.selected) / (len(widg.selected) + len(widg.ignored))
        print("Precision is {}".format(precision))
    calculate_precision_button.on_click(on_calculate)
    
    save_button = widgets.Button(
        layout=widgets.Layout(width='auto'),
        style=WIDGET_STYLE_ARGS,
        description='Save results',
        disabled=False,
        button_style='danger'
    )
    def on_save(b):
        with open(os.path.join(PATH, SAVE_FILE), 'wb') as f:
            pickle.dump({
                'positive': widg.selected,
                'negative': widg.ignored
            }, f)
            print('Results saved')
    save_button.on_click(on_save)
    
    display(calculate_precision_button)
    display(save_button)
    display(widg)
    display(calculate_precision_button)
    display(save_button)
    
    return widg

def get_precision(widg):
    return len(widg.selected) / (len(widg.selected) + len(widg.ignored))

print('Done!')

Loading libraries... Please wait.
Done!


# Init Constants

In [4]:
BASENAME = 'sampled_CLOSE_UP'

In [5]:
PATH = '/app/data/shot_scales_and_conv_idioms_samples'
FILE_NAME = '{}.pkl'.format(BASENAME)
START_INDEX = 0
END_INDEX = 100
SAVE_FILE = '{}_results_{}_{}.pkl'.format(BASENAME, START_INDEX, END_INDEX)

# Do labeling

Run the two cells below. `=` to expand frames - be sure to look at how long the clip is. `[` for positive examples, `]` for negative examples.

Shot scale definitions:
* Extreme long: the entire body of the character is visible and does not take up the whole frame.
* Long: the entire body is visible but fills the frame.
* Medium long:, the body up to the knee is visible.
* Medium: the body up to the waist is visible.
* Medium close ups, the body up to the chest is visible.
* In close ups, the head and shoulders are visible.
* Extreme close ups, only the face is visible (and not the neck).

Idiom definitions:
* Intensification: Series of shots in a conversation where the shot scale gets monotonically closer on at least one participant.
* Start wide: Conversation where the first shot is an establishing shot showing all the participants in the conversation.

In [6]:
esper_result = load_samples()

In [11]:
ignored = [6, 7, 8, 10, 22, 30, 42, 46, 59, 70, 76, 95]

In [12]:
selected = sorted(list(set(range(100)).difference(set(ignored))))

In [13]:
widg.ignored = ignored

In [14]:
widg.selected=selected

In [7]:
widg = label_segments(esper_result)

Button(description='Calculate Accuracy', layout=Layout(width='auto'), style=ButtonStyle())

Button(button_style='danger', description='Save results', layout=Layout(width='auto'), style=ButtonStyle())

VGridWidget(jsglobals={'schema': [['Identity', ['id', 'name']], ['Genre', ['id', 'name']], ['Director', ['id',…

Button(description='Calculate Accuracy', layout=Layout(width='auto'), style=ButtonStyle())

Button(button_style='danger', description='Save results', layout=Layout(width='auto'), style=ButtonStyle())

ZeroDivisionError: division by zero

Precision is 0.0
Precision is 0.88
Results saved


# Get Precision
Run the cell below to print the precision

In [15]:
get_precision(widg)

0.88